# PETs/TETs – Hyperledger Aries / PySyft – Manufacturer 2 (Holder) 💼

In [1]:
%%javascript
document.title = '💼 Manufacturer2 Agent'

<IPython.core.display.Javascript object>

## PART 3: Connect with City to Analyze Data

**What:** -

**Why:** -

**How:** <br>


**Accompanying Agents and Notebooks:**
* City 🏙️️: `03_connect_with_manufacturers.ipynb`

---

### 0 - Setup
#### 0.1 - Imports

In [2]:
import libs.helpers as helpers
import os
import syft as sy
import time
import torch

from aries_cloudcontroller import AriesAgentController
from libs.agent_connection_manager import CredentialHolder
from pprintpp import pprint
from termcolor import colored
from sympc.session import Session
from sympc.session import SessionManager
from sympc.tensor import MPCTensor

#### 0.2 – Variables

In [3]:
# Get relevant details from .env file
api_key = os.getenv("ACAPY_ADMIN_API_KEY")
admin_url = os.getenv("ADMIN_URL")
webhook_port = int(os.getenv("WEBHOOK_PORT"))
webhook_host = "0.0.0.0"

---

<a id=1></a>

### 1 – Initiate Manufacturer2 Agent
#### 1.1 – Init ACA-PY agent controller

In [4]:
# Setup
agent_controller = AriesAgentController(admin_url,api_key)
print(f"Initialising a controller with admin api at {admin_url} and an api key of {api_key}")

Initialising a controller with admin api at http://manufacturer2-agent:3021 and an api key of adminApiKey


#### 1.2 – Start Webhook Server to enable communication with other agents
@todo: is communication with other agents, or with other docker containers?

In [5]:
# Listen on webhook server
await agent_controller.init_webhook_server(webhook_host, webhook_port)
print(f"Listening for webhooks from agent at http://{webhook_host}:{webhook_port}")

Listening for webhooks from agent at http://0.0.0.0:3010


#### 1.3 – Init ACM issuing authority

In [6]:
# The CredentialHolder registers relevant webhook servers and event listeners
manufacturer2_agent = CredentialHolder(agent_controller)

# Verify if Manufacturer already has a VC
# (if there are manufacturer credentials, there is no need to execute the notebook)
manufacturer2_agent.get_credentials()

Successfully initiated AgentConnectionManager for a(n) Holder ACA-PY agent


{'results': [{'referent': 'M2-isManufacturer-VC',
   'attrs': {'manufacturerCountry': 'Germany',
    'manufacturerName': 'undisclosedManufacturer2',
    'manufacturerCity': 'Braunschweig',
    'isManufacturer': 'TRUE'},
   'schema_id': 'XvWddwCGryjykggMo3vHrv:2:certify-manufacturer:0.0.1',
   'cred_def_id': 'XvWddwCGryjykggMo3vHrv:3:CL:251651:default',
   'rev_reg_id': None,
   'cred_rev_id': None}]}

---

<a id=2></a>

### 2 – Establish a connection with the City agent 🏙️
A connection with the credential issuer (i.e., the authority agent) must be established before a VC can be received. In this scenario, the Manufacturer2 requests a connection with the Authority to be certified as an official city agency. Thus, the Manufacturer2 agent sends an invitation to the Authority. In real life, the invitation can be shared via video call, phone call, or E-Mail. In this PoC, this is represented by copy and pasting the invitation into the manufacturers' notebooks.

#### 2.1 Join invitation of City agent 🏙️
Copy and paste the multi-use invitation of the city agent, and establish a connection with them.

In [7]:
# Variables
alias = "undisclosedM2"
auto_accept = True
label = None

# Receive connection invitation
connection_id = manufacturer2_agent.receive_connection_invitation(alias=alias, auto_accept=auto_accept, label=label)

Please enter invitation received by external agent:


Invitation:  {     '@id': 'ac1eee2e-c63e-4415-a8a7-7ce5b434ea0f',     '@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/connections/1.0/invitation',     'label': 'City-Agency',     'recipientKeys': ['9wRd53Mahf746UnxEJgTDnyyRwYogTWEnxRFUSAj1LFA'],     'serviceEndpoint': 'https://0a6f916f7909.ngrok.io', }



---------------------------------------------------------------------
Connection Webhook Event Received: Connections Handler
Connection ID :  e8141e1d-bb54-4ea8-8c15-10984a00ea02
State :  invitation (invitation-received)
Routing State : none
Connection with :  City-Agency
Their Role :  inviter
---------------------------------------------------------------------

---------------------------------------------------------------------
Connection Webhook Event Received: Connections Handler
Connection ID :  e8141e1d-bb54-4ea8-8c15-10984a00ea02
State :  request (request-sent)
Routing State : none
Connection with :  City-Agency
Their Role :  inviter
---------------------------------------------------------------------

---------------------------------------------------------------------
Connection Webhook Event Received: Connections Handler
Connection ID :  e8141e1d-bb54-4ea8-8c15-10984a00ea02
State :  response (response-received)
Routing State : none
Connection with :  City-Agency
Their Ro

<div style="font-size: 25px"><center><b>Break Point 2/3/4</b></center></div>
<div style="font-size: 50px"><center>💼 ➡️ 💼 / 💼 / 🏙️ </center></div><br>
<center><b>Please proceed to the remaining Manufacturers. <br> If you have established a connection between the City and all Manufacturers, proceed to the City Notebook's Step 2.3</b></center>

---

### 3 – Create Presentation to Send Proof Presentation

#### 3.1 – Create presentation that satisfies requirements of proof request
Before you can present a presentation, you must identify the presentation record which you wish to respond to with a presentation. To do so, the `prepare_presentation()` function runs through the following steps: 
1. Get all proof requests that were sent through `connection_id`
2. Get the most recent `presentation_exchange_id` and the corresponding `proof_request` from (1)
3. Get the restrictions the City agent defined in `proof_request` from (2)
4. Compare all VCs the Manufacturer2 agent has stored, and find (if available) a VC that satisfies the restrictions from (3)
5. Return a presentation dictionary from a VC from (4) that satisfies all requirements. Generally, a presentation consists of three classes of attributes: <br>
a. `requested_attributes`: Attributes that were signed by an issuer and have been revealed in the presentation process <br>
b. `self_attested_attributes`: Attributes that the prover has self attested to in the presentation object. <br>
c. `requested_predicates` (predicate proofs): Attribute values that have been proven to meet some statement. (TODO: Show how you can parse this information)

In [8]:
presentation, presentation_exchange_id = manufacturer2_agent.prepare_presentation(connection_id)
presentation

> Found proof_request with presentation_exchange_id 8d1001f7-98ce-4284-b008-b928920ce26d
> Restrictions for a suitable proof: {'isManufacturer': {'requirements': {'schema_id': 'XvWddwCGryjykggMo3vHrv:2:certify-manufacturer:0.0.1'}, 'request_attr_name': '0_isManufacturer_uuid'}}
> Attribute request for 'isManufacturer' can be satisfied by Credential with VC 'M2-isManufacturer-VC'
> Generate the proof presentation


{'requested_predicates': {},
 'requested_attributes': {'0_isManufacturer_uuid': {'cred_id': 'M2-isManufacturer-VC',
   'revealed': True}},
 'self_attested_attributes': {}}

#### 3.2 – Send Presentation

Send the presentation to the recipient of `presentation_exchange_id`

In [9]:
manufacturer2_agent.send_proof_presentation(presentation_exchange_id, presentation)


---------------------------------------------------------------------
Connection Webhook Event Received: Present-Proof Handler
Connection ID :  e8141e1d-bb54-4ea8-8c15-10984a00ea02
Presentation Exchange ID :  8d1001f7-98ce-4284-b008-b928920ce26d
Protocol State :  presentation_sent
Agent Role :  prover
Initiator :  external
---------------------------------------------------------------------

---------------------------------------------------------------------
Connection Webhook Event Received: Present-Proof Handler
Connection ID :  e8141e1d-bb54-4ea8-8c15-10984a00ea02
Presentation Exchange ID :  8d1001f7-98ce-4284-b008-b928920ce26d
Protocol State :  presentation_acked
Agent Role :  prover
Initiator :  external
---------------------------------------------------------------------

Presentation Exchange ID: {pei} is acknowledged by Relying Party


<div style="font-size: 25px"><center><b>Break Point 6/7/8</b></center></div>
<div style="font-size: 50px"><center>💼 ➡️ 💼 / 💼 / 🏙️ </center></div><br>
<center><b>Please proceed to the remaining Manufacturers and run all cells between Steps 3 and 4.1 <br> If you have sent proof presentations from all manufacturers, proceed to the City Notebook's Step 3.3 </b></center>


---

### 4 – Do Data Science

#### 4.1 – Establish a Duet Connection: Send Duet invitation
Duet is a package that allows you to exchange encrypted data and run privacy-preserving arithmetic operations on them (e.g., through homomorphic encryption or secure multiparty computation).

In [12]:
# Set up connection_id to use for duet connection
manufacturer2_agent.set_duet_config(connection_id=connection_id)

# Create duet invitation for city agent
duet = sy.launch_duet(credential_exchanger=manufacturer2_agent)

🎤  🎸  ♪♪♪ Starting Duet ♫♫♫  🎻  🎹

♫♫♫ > DISCLAIMER: Duet is an experimental feature currently in beta.
♫♫♫ > Use at your own risk.


    > ❤️ Love Duet? Please consider supporting our community!
    > https://github.com/sponsors/OpenMined

♫♫♫ > Punching through firewall to OpenGrid Network Node at:
♫♫♫ > http://ec2-18-218-7-180.us-east-2.compute.amazonaws.com:5000
♫♫♫ >
♫♫♫ > ...waiting for response from OpenGrid Network... 
♫♫♫ > DONE!
♫♫♫ > Duet Server ID: 9dd6874a4fd57d968bf8809c1b316699

♫♫♫ > STEP 1: Send the following code to your Duet Partner!

import syft as sy
duet = sy.duet("9dd6874a4fd57d968bf8809c1b316699")

♫♫♫ > STEP 2: Ask your partner for their Client ID and enter it below!


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/opt/conda/lib/python3.9/site-packages/IPython/core/interactiveshell.py", line 3441, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_15/4227218017.py", line 2, in <module>
    duet = sy.duet()
  File "/opt/conda/lib/python3.9/site-packages/syft/grid/duet/__init__.py", line 167, in duet
    return launch_duet(
  File "/opt/conda/lib/python3.9/site-packages/syft/grid/duet/__init__.py", line 217, in launch_duet
    target_id = credential_exchanger.run(credential=signaling_client.duet_id)
  File "/opt/conda/lib/python3.9/site-packages/syft/grid/duet/exchange_ids.py", line 40, in run
    return self._server_exchange(credential=self.credential)
  File "/opt/conda/lib/python3.9/site-packages/syft/grid/duet/exchange_ids.py", line 74, in _server_exchange
    client_id = input("♫♫♫ > Duet Partner's Client ID: ")  # nosec
  File "/opt/conda/lib/python3.9/site-packages/ipykernel/kernelbase.py", line 981, in raw_inp

TypeError: object of type 'NoneType' has no len()

## 2 - Secure Multi-Party Computation

In [ ]:
# Step 8: test there is no data in the server atm
duet_m1.store.pandas

In [ ]:
age_data = th.tensor([50,23,72,83])
age_data = age_data.tag("test_data_age")
age_data = age_data.describe("description for age_data")
age_data_pointer = age_data.send(duet_m1, pointable=True)

In [ ]:
duet_m1.store

In [ ]:
duet_m1.store.pandas

In [ ]:
duet_m1.id

In [ ]:
duet_m1.name

In [ ]:
duet_m1.target_id

In [ ]:
duet_m1.network_id

In [ ]:
#duet_city.store.pandas

In [ ]:
# Step 9: Publish secret data

x = torch.tensor([50,60,77]) # define torch
x.send(duet_m1, pointable=True, tags=["TEST_#1_from_m1_to_ds"], description="Dummy data") # send torch to duet_m1

x = torch.tensor([[1,2], [3,4]])
x.send(duet_m1, pointable=True, tags=["TEST_#2_from_m1_to_ds"], description="Test to multiply matrices")

In [ ]:
# Step 10: verify that data was uploaded to the store
duet_m1.store.pandas

In [ ]:
# Step 11: Authorize the requests ds can make
duet_m1.requests.add_handler(action="accept")

In [ ]:
# Step 11: Authorize the requests ds can make
duet_city.requests.add_handler(action="accept")

## Terminate Controller

Whenever you have finished with this notebook, be sure to terminate the controller. This is especially important if your business logic runs across multiple notebooks.

In [9]:
await agent_controller.terminate()